<a href="https://colab.research.google.com/github/panghanwu/scikit-learning_tutorial_Elwing/blob/main/naive_bayes_multinomial_poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 唐詩預測

## 不適合使用決策樹
1. 維度災難(欄位太多，多於數百)，演算法可能會崩潰。
2. 欄位越多會需要更多的資料(資料疏密度)。

## 機率
$
P(A):A發生的機率\\
P(A,B),P(A\cap B):A,B同時發生的機率(交集)\\
P(A|B)\ or\ P(\frac{A}{B}):B條件下,A發生的機率\\
P(A,B)=P(A)P(B|A)=P(B)P(A|B)-(1)\\
P(A,B,C)=P(A)P(B|A)P(C|A,B)
$
## 貝氏定理
$
from\ (1)\\
P(A)P(B|A)=P(B)(A|B)\\
P(A|B)=\dfrac{P(A)}{P(B)}P(B|A)
$
## 獨立事件
$
P(A)P(A|B)=P(A)P(B)
$
## 舉例
$
P(\dfrac{李白}{月光}):詩詞出現"月光"，作者是"李白"的機率。\\
P(李白):這首詩作者是"李白"的機率。\\
P(\dfrac{月光}{李白}):"李白"寫出"月光"的機率\\
P(\dfrac{李白}{月光})=P(李白)\times P(\dfrac{月光}{李白})
$
## 非獨立事件
$
P(\dfrac{床前,明,月光}{李白})=P(\dfrac{床前}{李白})P(\dfrac{明,月光}{李白,床前})P(\dfrac{月光}{李白,床前,明})
$
## 單純貝氏(以獨立事件近似)
$
P(\dfrac{床前,明,月光}{李白})=P(\dfrac{床前}{李白})P(\dfrac{明}{李白})P(\dfrac{月光}{李白})
$
- 寫詩用字比喻成丟骰子
- 有效解決維度災難
- 適合用在語言的問題
- 可以用少量的資料得到不錯的訓練結果


In [ ]:
from urllib.request import urlretrieve

url = 'https://github.com/Elwing-Chou/ml0930/raw/master/poem_train.csv'
urlretrieve(url, 'train.csv')
url = 'https://github.com/Elwing-Chou/ml0930/raw/master/poem_test.csv'
urlretrieve(url, 'test.csv')


('test.csv', <http.client.HTTPMessage at 0x7f187fab96a0>)

In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv', encoding='utf-8')
train_df

作者  ...                                                 內容
0      李白  ...  平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。\r...
1      李白  ...  青天有月來幾時，我今停杯一問之：人攀明月不可得，月行卻與人相隨？皎如飛鏡臨丹闕，綠煙滅儘清輝...
2      李白  ...               燕草如碧絲，秦桑低綠枝。當君懷歸日，是妾斷腸時。春風不相識，何事入羅幃。
3      李白  ...                   誰家玉笛暗飛聲，散入春風滿洛城。此夜曲中聞折柳，何人不起故園情。
4      李白  ...  西上蓮花山，迢迢見明星。(西上 一作：西嶽)素手把芙蓉，虛步躡太清。霓裳曳廣帶，飄拂升天行。...
...   ...  ...                                                ...
2726  白居易  ...  零落桐葉雨，蕭條槿花風。悠悠早秋意，生此幽閒中。況與故人彆，中懷正無悰。勿雲不相送，心到青門...
2727  白居易  ...  離離暑雲散，嫋嫋涼風起。池上秋又來，荷花半成子。朱顏易銷歇，白日無窮已。人壽不如山，年光急於...
2728  白居易  ...  涼風冷露蕭索天，黃蒿紫菊荒涼田。繞塚秋花少顏色，細蟲小蝶飛翻翻。中有騰騰獨行者，手拄漁竿不騎...
2729  白居易  ...  往歲曾為西邑吏，慣從駱口到南秦。\r\n三時雲冷多飛雪，二月山寒少有春。\r\n我思舊事猶惆...
2730  白居易  ...  笛竹出蘄春，霜刀劈翠筠。織成雙鎖簟，寄與獨眠人。卷作筒中信，舒為席上珍。滑如鋪薤葉，冷似臥龍...

[2731 rows x 3 columns]

In [ ]:
test_df = pd.read_csv('test.csv', encoding='utf-8')
test_df

作者  ...                                                 內容
0    李白  ...                   日照香爐生紫煙，遙看瀑布掛前川。飛流直下三千尺，疑是銀河落九天。
1    李白  ...                   朝辭白帝彩雲間，千裡江陵一日還。兩岸猿聲啼不住，輕舟已過萬重山。
2    李白  ...                   李白乘舟將欲行，忽聞岸上踏歌聲。桃花潭水深千尺，不及汪倫送我情。
3    李白  ...                   故人西辭黃鶴樓，煙花三月下揚州。孤帆遠影碧空儘，唯見長江天際流。
4    李白  ...                           危樓高百尺，手可摘星辰。不敢高聲語，恐驚天上人。
5    李白  ...                           床前明月光，疑是地上霜。舉頭望明月，低頭思故鄉。
6    李白  ...                   天門中斷楚江開，碧水東流至此回。兩岸青山相對出，孤帆一片日邊來。
7    李白  ...                           眾鳥高飛儘，孤雲獨去閒。相看兩不厭，隻有敬亭山。
8    李白  ...  鳳凰台上鳳凰遊，鳳去台空江自流。吳宮花草埋幽徑，晉代衣冠成古丘。三山半落青天外，二水中分白鷺...
9    李白  ...   渡遠荊門外，來從楚國遊。山隨平野儘，江入大荒流。月下飛天境，雲生結海樓。仍憐故鄉水，萬裡送行舟。
10   杜甫  ...  百川日東流，客去亦不息。我生苦漂蕩，何時有終極。讚公釋門老，放逐來上國。還為世塵嬰，頗帶憔悴...
11   杜甫  ...  細泉兼輕冰，沮洳棧道濕。不辭辛苦行，迫此短景急。石門雪雲隘，古鎮峰巒集。旌竿暮慘澹，風水白刃...
12   杜甫  ...  首路栗亭西，尚想鳳凰村。季冬攜童稚，辛苦赴蜀門。南登木皮嶺，艱險不易論。汗流被我體，祁寒為之...
13   杜甫  ...   落日在簾鉤，溪邊春事幽。芳菲緣岸圃，樵爨倚灘舟。啅雀爭枝墜，飛蟲滿院遊。濁醪誰造汝，一酌散千憂。
14   杜甫  ...  竇侍禦，驥之子，鳳之雛。年未三十忠義俱，骨鯁絕代無。炯如一段清冰出萬壑，置在迎風寒露之玉壺。...
15   杜甫  ...   百草競春華，麗春應最勝。少須好顏色，多漫枝條剩。紛紛桃李枝，處處總能移。如何貴此重，卻怕有人知。
16   杜甫  ...   何年顧虎頭，滿壁畫瀛州。赤日石林氣，青天江海流。錫飛常近鶴，杯度不驚鷗。似得廬山路，真隨惠遠遊。
17   杜甫  ...  野寺隱喬木，山僧高下居。石門日色異，絳氣橫扶疏。窈窕入風磴，長蘆紛卷舒。庭前猛虎臥，遂得文公...
18   杜甫  ...  西川有杜鵑，東川無杜鵑。涪萬無杜鵑，雲安有杜鵑。我昔遊錦城，結廬錦水邊。有竹一頃餘，喬木上參...
19   杜甫  ...  小奴縛雞向市賣，雞被縛急相喧爭。家中厭雞食蟲蟻，不知雞賣還遭烹。蟲雞於人何厚薄，我斥奴人解其...
20  白居易  ...                   管妙弦清歌入雲，老人合眼醉醺醺。誠知不及當年聽，猶覺聞時勝不聞。
21  白居易  ...  追歡逐樂少閒時，補帖平生得事遲。何處花開曾後看？誰家酒熟不先知？石樓月下吹蘆管，金穀風前舞柳...
22  白居易  ...  歲陰生計兩蹉跎，相顧悠悠醉且歌。廚冷難留烏止屋，門閒可與雀張羅。[詩雲：贍烏爰止，於誰之屋？...
23  白居易  ...  雨砌長寒蕪，風庭落秋果。窗間有閒叟，儘日看書坐。書中見往事，曆曆知福禍。多取終厚亡，疾驅必先...
24  白居易  ...  睡足肢體暢，晨起開中堂。初旭泛簾幕，微風拂衣裳。二婢扶盥櫛，雙童舁簟床。庭東有茂樹，其下多陰...
25  白居易  ...  履道西門有弊居，池塘竹樹繞君廬。豪華肥壯雖無分，飽暖安閒即有餘。行灶朝香炊早飯，小園春暖掇新...
26  白居易  ...  昨日複今辰，悠悠七十春。所經多故處，卻想似前身。散秩優遊老，閒居淨潔貧。螺杯中有物，鶴氅上無...
27  白居易  ...  不與老為期，因何兩鬢絲？才應免夭促，便已及衰羸。昨夜夢何在，明朝身不知。百憂非我所，三樂是吾...
28  白居易  ...                   暖床斜臥日曛腰，一覺閒眠百病銷。儘日一飧茶兩碗，更無所要到明朝。
29  白居易  ...  選石鋪新路，安橋壓古堤。似從銀漢下，落傍玉川西。影定欄杆倒，標高華表齊。煙開虹半見，月冷鶴雙...

[30 rows x 3 columns]

In [ ]:
# Serise.replace({'李白':0})
# unique/value_counts
writers = train_df['作者'].unique()
writers

array(['李白', '杜甫', '白居易'], dtype=object)

In [ ]:
result = {}
for i, w in enumerate(writers):
  result[w] = i
result

{'李白': 0, '杜甫': 1, '白居易': 2}

In [ ]:
trans = {w:i for i, w in enumerate(writers)}
trans

{'李白': 0, '杜甫': 1, '白居易': 2}

In [ ]:
trans_r = {i:w for i, w in enumerate(writers)}
trans_r

{0: '李白', 1: '杜甫', 2: '白居易'}

In [ ]:
y_train = train_df['作者'].replace(trans)
y_test = test_df['作者'].replace(trans)
y_test

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    2
21    2
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    2
Name: 作者, dtype: int64

In [ ]:
type(train_df['內容'])

pandas.core.series.Series

In [ ]:
p = train_df['內容'][0]
p

'平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。\r\n何處是歸程？長亭連短亭。(連短亭 一作：更短亭) \r\n        '

In [ ]:
import jieba

' '.join(jieba.cut(p))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.024 seconds.
Prefix dict has been built successfully.


'平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 \r\n 暝 色入 高樓 ， 有人 樓上 愁 。 玉階空 佇立 ， 宿鳥 歸 飛急 。 \r\n 何處 是 歸程 ？ 長 亭 連短亭 。 ( 連短亭   一作 ： 更 短亭 )   \r\n                '

## function() 
function: 步驟的集合  
(): 執行

In [ ]:
def test():
  return int
test()(5.3)

5

In [ ]:
# apply(函式名字)
def poemcut(p):
  return ' '.join(jieba.cut(p))
x_train = train_df['內容'].apply(poemcut)
x_test = test_df['內容'].apply(poemcut)
x_test

0      日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1     朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2      李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3       故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4                 危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。
5                  床前 明月光 ， 疑是 地上 霜 。 舉頭 望明月 ， 低頭思 故鄉 。
6     天門 中斷 楚江 開 ， 碧水 東流 至此 回 。 兩岸 青山 相對 出 ， 孤帆 一片 日...
7               眾鳥 高 飛 儘 ， 孤雲獨 去 閒 。 相看 兩不厭 ， 隻 有 敬亭山 。
8     鳳凰 台上 鳳凰遊 ， 鳳去 台空江 自流 。 吳宮 花草 埋 幽徑 ， 晉代 衣冠 成古丘...
9     渡遠 荊門外 ， 來 從 楚國遊 。 山 隨 平野 儘 ， 江入 大荒 流 。 月 下 飛天...
10    百川 日東流 ， 客去 亦 不息 。 我 生苦 漂 蕩 ， 何時 有 終極 。 讚 公 釋 ...
11    細泉 兼 輕冰 ， 沮洳 棧道 濕 。 不辭 辛苦 行 ， 迫此 短景急 。 石門 雪雲隘 ...
12    首路 栗亭 西 ， 尚想 鳳凰村 。 季冬 攜 童稚 ， 辛苦 赴 蜀門 。 南登 木皮 嶺...
13    落日 在 簾 鉤 ， 溪邊 春事幽 。 芳菲 緣岸 圃 ， 樵 爨 倚灘 舟 。 啅 雀 爭...
14    竇侍 禦 ， 驥 之子 ， 鳳之雛 。 年 未 三十 忠義俱 ， 骨鯁 絕代 無 。 炯 如...
15    百草 競春華 ， 麗春應 最勝 。 少須 好 顏色 ， 多漫枝條 剩 。 紛紛 桃李 枝 ，...
16    何年 顧虎頭 ， 滿壁畫 瀛州 。 赤 日 石林 氣 ， 青天 江海 流 。 錫飛常 近鶴 ...
17    野寺 隱喬木 ， 山僧 高下 居 。 石門 日色異 ， 絳 氣橫 扶疏 。 窈窕 入

fit(raw_documents[, y])/fit_transform(raw_documents[, y])

train: fit+transform

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
x_train_count
# sparse: 只存有值的欄位，只有85677筆

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

In [ ]:
print(x_train_count)

  (0, 16053)	1
  (0, 29006)	1
  (0, 30177)	1
  (0, 14002)	1
  (0, 139)	1
  (0, 4756)	1
  (0, 39345)	1
  (0, 51400)	1
  (0, 23289)	1
  (0, 25180)	1
  (0, 31382)	1
  (0, 3549)	1
  (0, 13846)	1
  (0, 50568)	1
  (0, 3797)	1
  (0, 26005)	1
  (0, 44987)	2
  (0, 34)	1
  (0, 33975)	1
  (1, 49338)	1
  (1, 16293)	1
  (1, 19165)	1
  (1, 4540)	1
  (1, 85)	1
  (1, 2776)	1
  :	:
  (2729, 8800)	1
  (2729, 6485)	1
  (2729, 3060)	1
  (2730, 25782)	1
  (2730, 6026)	1
  (2730, 36958)	1
  (2730, 15905)	1
  (2730, 44854)	1
  (2730, 29458)	1
  (2730, 2175)	1
  (2730, 31125)	1
  (2730, 40974)	1
  (2730, 35446)	1
  (2730, 49196)	1
  (2730, 6770)	1
  (2730, 48596)	1
  (2730, 7813)	1
  (2730, 1714)	1
  (2730, 39190)	1
  (2730, 28820)	1
  (2730, 38489)	1
  (2730, 28423)	1
  (2730, 51689)	1
  (2730, 8048)	1
  (2730, 23226)	1


In [ ]:
print(x_test_count)

  (0, 21652)	1
  (0, 29325)	1
  (0, 32404)	1
  (0, 33259)	1
  (0, 36144)	1
  (0, 40411)	1
  (0, 45388)	1
  (0, 46901)	1
  (0, 50588)	1
  (1, 201)	1
  (1, 1037)	1
  (1, 5321)	1
  (1, 7398)	1
  (1, 15739)	1
  (1, 23671)	1
  (1, 26945)	1
  (1, 30993)	1
  (1, 32734)	1
  (1, 44300)	1
  (1, 46477)	1
  (2, 1092)	1
  (2, 2160)	1
  (2, 7343)	1
  (2, 15312)	1
  (2, 17635)	1
  :	:
  (26, 18010)	1
  (26, 19510)	1
  (26, 19833)	1
  (26, 22302)	1
  (26, 23398)	1
  (26, 24210)	1
  (26, 29775)	1
  (26, 51930)	1
  (27, 16)	1
  (27, 1131)	1
  (27, 1361)	1
  (27, 3576)	1
  (27, 3638)	1
  (27, 5425)	1
  (27, 10217)	1
  (27, 12065)	1
  (27, 14735)	1
  (27, 21864)	1
  (27, 22290)	1
  (27, 22338)	1
  (27, 47253)	1
  (28, 415)	1
  (28, 21864)	1
  (28, 47455)	1
  (29, 50837)	1


In [ ]:
vec.vocabulary_

In [ ]:
vec.vocabulary_['。']
# no "。" in columns

KeyError: ignored

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.5)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

## Smomthing
$
\theta = \dfrac{N_i+\alpha}{N+\alpha n}
$

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8666666666666667

In [ ]:
from sklearn.metrics import confusion_matrix
# 混淆矩陣
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      index=[name + '(原本)' for name in writers],
      columns=[name + '(預測)' for name in writers])

李白(預測)  杜甫(預測)  白居易(預測)
李白(原本)        8       1        1
杜甫(原本)        0      10        0
白居易(原本)       1       1        8

In [ ]:
list(zip([0,2,3],[4,5,6]))

[(0, 4), (2, 5), (3, 6)]

In [ ]:
p = input('輸入一首詩:')
x_predict = vec.transform([poemcut(p)])
proba = clf.predict_proba(x_predict)[0]
for w, p in zip(writers, proba):
  print(w, ':', p)

輸入一首詩:君不見 黃河之水天上來
李白 : 0.8573778616749936
杜甫 : 0.1068457653457308
白居易 : 0.035776372979274405
